In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset
import utils



class CustomDataset(Dataset):
    def __init__(self, features, targets):
        self.features = torch.tensor(features, dtype=torch.float32)
        self.targets = torch.tensor(targets, dtype=torch.long)

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return self.features[idx], self.targets[idx]


def scale_column(x_train:pd.DataFrame, x_test:pd.DataFrame, column:list):
    """
    Move to utils_NN.py
    Scales the same nature columns of x_train and x_test using the MinMaxScaler. The reference column is the one with the maximum value.


    """
    max_value = x_train[column].max().max()
    x_train[column] = x_train[column]/max_value
    x_test[column]  = x_test[column]/max_value
    return x_train, x_test, max_value

pygame 2.6.1 (SDL 2.28.4, Python 3.11.10)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
# Parameters
model_type =  'classifier' # 'classifier' or 'regressor'
use_trap_info = True
ntraps = 3
lags = 3
random_split = True
test_size = 0.2
scale = True
learning_rate = 1e-3
batch_size = 64
epochs = 10

In [3]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


In [4]:
# data import and preprocessing
data = pd.read_csv(f'./results/final_df_lag{lags}_ntraps{ntraps}.csv')
n = data.shape[0]
nplaca_index = data['nplaca']
data.drop(columns=['nplaca','distance0'], inplace=True) # drop distance0 because it is always zero
ovos_flag = data['novos'].apply(lambda x: 1 if x > 0 else 0)

# divide columns into groups
days_columns = [f'days{i}_lag{j}' for i in range(ntraps) for j in range(1, lags+1)]
distance_columns = [f'distance{i}' for i in range(1,ntraps)]
eggs_columns = [f'trap{i}_lag{j}' for i in range(ntraps) for j in range(1, lags+1)]


In [5]:
# definition of x and y
if model_type == 'classifier':
    y = ovos_flag
else:
    y = data['novos']
if use_trap_info:
    x = data.drop(columns=['novos'])
else:
    drop_cols = ['novos'] + days_columns + distance_columns
    x = data.drop(columns=drop_cols)

In [6]:
# train test split
train_size = 1 - test_size

if random_split:
    x_train, x_test, y_train, y_test = train_test_split(data.drop(columns=['novos']), y, test_size=test_size, random_state=42, stratify=ovos_flag)
else:
    y_train = y.iloc[:int(n*train_size)]
    y_test = y.iloc[int(n*train_size):]
    x_train = x.iloc[:int(n*train_size)]
    x_test = x.iloc[int(n*train_size):]    

In [7]:
# scaling
x_train, x_test, max_eggs = scale_column(x_train, x_test, eggs_columns)
if use_trap_info:
    x_train, x_test, max_distance = scale_column(x_train, x_test, distance_columns)
    x_train, x_test, max_days = scale_column(x_train, x_test, days_columns)


if model_type != 'classifier':
    y_train = y_train/max_eggs
    y_test = y_test/max_eggs

In [8]:
# transform to tensors
xtrain = torch.tensor(x_train.values, dtype=torch.float32).to(device)
ytrain = torch.tensor(y_train.values, dtype=torch.long).to(device)
xtest = torch.tensor(x_test.values, dtype=torch.float32).to(device)
ytest = torch.tensor(y_test.values, dtype=torch.long).to(device)

train_dataset = CustomDataset(xtrain, ytrain)
test_dataset = CustomDataset(xtest, ytest)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=random_split)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=random_split)

C:\Users\User\AppData\Local\Temp\ipykernel_6884\450902588.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.features = torch.tensor(features, dtype=torch.float32)
C:\Users\User\AppData\Local\Temp\ipykernel_6884\450902588.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.targets = torch.tensor(targets, dtype=torch.long)


In [9]:
# Network structure
if use_trap_info:
    model_input = lags*ntraps + ntraps-1 + ntraps*lags # sum  of eggs, distances minus one and days
else:
    model_input = lags*ntraps
    
if model_type == 'classifier':
    model_output = 2


class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.structure = nn.Sequential(
            nn.Linear(model_input, 20),
            nn.Sigmoid(),
            nn.Linear(20, 10),
            nn.Sigmoid(),
            nn.Linear(10, 5),
            nn.Sigmoid(),
            nn.Linear(5, model_output)

        )

    def forward(self, x):
        logits = self.structure(x)
        return logits    

In [10]:
# Create train and test loops
def train_loop(dataloader, model, loss_fn, optimizer,batch_size):
    size = xtrain.shape[0]
    model.train()
    for batch, (xtest, ytest) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(xtest)
        loss = loss_fn(pred, ytest)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * batch_size + len(xtest)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Metrics: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [11]:
model = NeuralNetwork().to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)


loss_hist = []
accuracy_hist = []
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer, batch_size)
    loss_hist.append(loss_fn(model(xtrain), ytrain).item())
    test_loop(test_dataloader, model, loss_fn)
    accuracy_hist.append((model(xtest).argmax(1) == ytest).type(torch.float).sum().item()/len(ytest))
    torch.save(model.state_dict(), f'./results/NN/save_parameters/model_lags{lags}_ntraps{3}_epoch{t}.pth')
    
    
print("Done!")
utils.play_ending_song()
utils.stop_ending_song()

Epoch 1
-------------------------------
loss: 0.694440  [   64/267656]
loss: 0.698057  [ 6464/267656]
loss: 0.696385  [12864/267656]
loss: 0.693171  [19264/267656]
loss: 0.685402  [25664/267656]
loss: 0.688050  [32064/267656]
loss: 0.692805  [38464/267656]
loss: 0.679905  [44864/267656]
loss: 0.677932  [51264/267656]
loss: 0.676404  [57664/267656]
loss: 0.685993  [64064/267656]
loss: 0.673894  [70464/267656]
loss: 0.682259  [76864/267656]
loss: 0.675038  [83264/267656]
loss: 0.696304  [89664/267656]
loss: 0.659471  [96064/267656]
loss: 0.666422  [102464/267656]
loss: 0.700801  [108864/267656]
loss: 0.705110  [115264/267656]
loss: 0.664731  [121664/267656]
loss: 0.672869  [128064/267656]
loss: 0.672604  [134464/267656]
loss: 0.689705  [140864/267656]
loss: 0.676588  [147264/267656]
loss: 0.694563  [153664/267656]
loss: 0.685538  [160064/267656]
loss: 0.690279  [166464/267656]
loss: 0.699673  [172864/267656]
loss: 0.657429  [179264/267656]
loss: 0.676139  [185664/267656]
loss: 0.642855  

In [13]:
y_test.sum()/len(y_test)

np.float64(0.42055474190752307)